# Plot n-dimensional distributions

In [ ]:
import os 
import sys

import numpy as np
import proplot as pplt
import psdist as ps
import psdist.visualization as psv
import scipy.ndimage

import mentflow as mf

In [ ]:
pplt.rc["cmap.discrete"] = False
pplt.rc["cmap.sequential"] = "viridis"
pplt.rc["figure.facecolor"] = "white"
pplt.rc["grid"] = False

In [ ]:
d = 6
size = int(1.00e+06)
normalize = True
n_bins = 80
rmax = 4.25

seed = 0
rng = np.random.default_rng(seed)

In [ ]:
for name, constructor in mf.dist.dist_nd.DISTRIBUTIONS.items():
    print(name)
        
    dist = constructor(d=d, normalize=normalize, noise=None, rng=rng)
    X = dist.sample(size)
    X = X.detach().cpu().numpy()
    
    grid = psv.CornerGrid(d=d, diag_rspine=False, figwidth=((5.0 / 4.0) * d))
    grid.plot_points(X, limits=(d * [(-rmax, rmax)]), bins=55, mask=False, cmap="mono")
    pplt.show()
    
    r = np.linalg.norm(X, axis=1)

    hist, edges = ps.points.radial_histogram(X, bins=100, limits=(0.0, rmax))
    hist = scipy.ndimage.gaussian_filter(hist, 1.0)
    
    fig, ax = pplt.subplots(figsize=(3.0, 2.0))
    ax.stairs(hist, edges, color="black", lw=1.25)
    ax.format(xlabel="Radius", ylabel="Normalized density")
    pplt.show()

    hist, edges = np.histogram(r, bins=50, range=(0.0, rmax))

    fig, ax = pplt.subplots(figsize=(3.0, 2.0))
    ax.stairs(hist, edges, color="black", lw=1.25)
    ax.format(xlabel="Radius", ylabel="Unnormalized density")
    pplt.show()

In [ ]:
dist = mf.dist.dist_nd.gen_dist(name="rings", d=6)

X = dist.sample(int(1.00e+06))
X = X.detach().cpu().numpy()


from ipywidgets import interact
@interact(width=(0.0, 10.0), center=(-3.0, 3.0), bins=(10, 150))
def update(width, center, bins):
    Y = ps.points.slice_planar(
        X, axis=list(range(1, d)), center=center, width=width,
    )
    
    fig, ax = pplt.subplots()
    ax.hist(Y[:, 0], bins=bins, range=(-4.5, 4.5), color="black")
    pplt.show()

In [ ]:
# psv.points.proj1d_interactive_slice(X, color="black")